In [1]:
import numpy as np
import matplotlib.pylab as plt
import time

class Swarm:
    def __init__(self):
        self.w = 0.5
        self.lb = -4.5
        self.ub = 4.5
        self.c1 = 2.5
        self.c2 = 1.5
        self.PARTICLE_NUMBER = 1000
        self. ITERATION_NUMBER = 100
        self.particle_history = []

    def _function(self, particles_pos):
        return (1.5 - particles_pos[0] - particles_pos[0] * particles_pos[1])**2 + \
            (2.25 - particles_pos[0] + particles_pos[0]*particles_pos[1]**2)**2 \
            + (2.625 - particles_pos[0] + particles_pos[0] * particles_pos[1] ** 3)**2

    def check_bounds(self, particles_pos):
        clipped_array = np.clip(particles_pos, self.lb, self.ub)
        return np.array_equal(particles_pos, clipped_array)
    

    def update_plot(self):
        plt.clf()
        plt.figure(figsize=(8, 6))
        for i in range(len(self.particle_history)):
            plt.scatter(self.particle_history[i][:, 0], self.particle_history[i][:, 1], color='red', marker='o', alpha=min(1, i / len(self.particle_history)))
        plt.xlim(self.lb, self.ub)
        plt.ylim(self.lb, self.ub)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('Particle Swarm Optimization')
        plt.pause(0.1)

    def pso(self):
        particles_pos = np.random.uniform(self.lb, self.ub, size=(self.PARTICLE_NUMBER, 2))
        particles_vel = np.zeros((self.PARTICLE_NUMBER, 2))
        personal_best_pos = particles_pos.copy()
        personal_best_val = np.array([self._function(pb) for pb in personal_best_pos])

        global_best_pos = personal_best_pos[np.argmin(personal_best_val)]
        global_best_val = np.min(personal_best_val)

        for _ in range(self.ITERATION_NUMBER):
            for i in range(self.PARTICLE_NUMBER):
                particles_vel[i] = self.w * particles_vel[i] + self.c1 * np.random.rand(2) * (personal_best_pos[i] - particles_pos[i]) + self.c2 * np.random.rand(2) * (global_best_pos - particles_pos[i])
                
                particles_pos[i] += particles_vel[i]
                
                if not self.check_bounds(particles_pos[i]):
                    particles_pos[i] = np.clip(particles_pos[i], self.lb, self.ub)
                    particles_vel[i] = -particles_vel[i]
                
                val = self._function(particles_pos[i])
                if val < personal_best_val[i]:
                    personal_best_val[i] = val
                    personal_best_pos[i] = particles_pos[i]
                    
                    if val < global_best_val:
                        global_best_val = val
                        global_best_pos = particles_pos[i]
            self.particle_history.append(particles_pos.copy())
        return global_best_pos, global_best_val, self.particle_history

In [2]:
from matplotlib.animation import FuncAnimation

def update_plot(frame, particle_history, sc):
    sc.set_offsets(particle_history[frame])
    return sc,

In [3]:
swarm = Swarm()
best_pos, best_val, particle_history = swarm.pso()        
print(f'Best pos: {best_pos} for value: {best_val}')

lb = -4.5
ub = 4.5
fig, ax = plt.subplots()
ax.set_xlim(lb, ub)
ax.set_ylim(lb, ub)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Particle Swarm Optimization')
sc = ax.scatter([], [], color='red', marker='o')

ani = FuncAnimation(fig, update_plot, frames=len(particle_history), fargs=(particle_history, sc), blit=True, interval=100)

plt.close()
from IPython.display import HTML
HTML(ani.to_jshtml())

Best pos: [ 2.51799777 -0.37462965] for value: 0.013514494946178715
